# Preamble

Ignore the errors raised because of nb-black

In [ ]:
cd ../

In [7]:
pwd

'/Users/ashkanaleali/w/infer_struct'

time: 2.4 ms (started: 2021-03-30 22:50:53 -05:00)


In [8]:
%load_ext lab_black
%matplotlib inline
%load_ext autoreload
%autoreload 2
%load_ext autotime

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 7.92 ms (started: 2021-03-30 22:51:28 -05:00)


## Imports

In [59]:
import os
import numpy as np
import pandas as pd

time: 2.92 ms (started: 2021-03-30 23:21:34 -05:00)


In [109]:
from infer_struct.data import PowerGridSimpleExample

time: 4.13 ms (started: 2021-03-31 00:12:57 -05:00)


# Data

This notebook is just to see how to load the data Amir has provided.

In [110]:
node_number = 9
data_dir = "../data/Network_raw_data_Ashkan/"
example_id = f"{node_number}_nodes"

time: 5.06 ms (started: 2021-03-31 00:12:59 -05:00)


In [114]:
ex = PowerGridSimpleExample(node_number)

time: 27.8 ms (started: 2021-03-31 00:15:38 -05:00)


+ There are 2 $nxn$ matrices and 6 vectors for each example. The vectors have length < $n$ and the missing elements are treated as zero.
+ Each vector is given by a tuple where the first element is the index of the element -- sparse matrix representation?


In [ ]:
np.fromiter()

In [134]:
print(np.pad(ex.Ybus_mag, (1, 0)))

[[ 0.      0.      0.      0.      0.      0.      0.      0.      0.
   0.    ]
 [ 0.      0.     16.      0.      0.      0.      0.     16.      0.
   0.    ]
 [ 0.      0.      0.     17.065   0.      0.      0.      0.      0.
  17.065 ]
 [ 0.     17.361   0.      0.     39.614  11.684  10.689   0.      0.
   0.    ]
 [ 0.      0.      0.      0.     11.684  17.764   0.      6.092   0.
   0.    ]
 [ 0.      0.      0.      0.     10.689   0.     16.419   0.      0.
   5.7334]
 [ 0.      0.     16.      0.      0.      6.092   0.     35.783  13.793
   0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.     13.793  23.645
   9.8522]
 [ 0.      0.      0.     17.065   0.      0.      5.7334  0.      9.8522
  32.529 ]]
time: 4.64 ms (started: 2021-03-31 00:24:28 -05:00)


In [147]:
print(np.vstack((np.zeros_like(ex.Ybus_mag[0]), ex.Ybus_mag)))

[[ 0.      0.      0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.     16.      0.      0.      0.      0.     16.      0.      0.    ]
 [ 0.      0.     17.065   0.      0.      0.      0.      0.     17.065 ]
 [17.361   0.      0.     39.614  11.684  10.689   0.      0.      0.    ]
 [ 0.      0.      0.     11.684  17.764   0.      6.092   0.      0.    ]
 [ 0.      0.      0.     10.689   0.     16.419   0.      0.      5.7334]
 [ 0.     16.      0.      0.      6.092   0.     35.783  13.793   0.    ]
 [ 0.      0.      0.      0.      0.      0.     13.793  23.645   9.8522]
 [ 0.      0.     17.065   0.      0.      5.7334  0.      9.8522 32.529 ]]
time: 6.93 ms (started: 2021-03-31 00:27:45 -05:00)


In [133]:
ex.Ybus_mag.shape

(8, 9)

time: 4.04 ms (started: 2021-03-31 00:23:54 -05:00)


In [138]:
np.pad(ex.Ybus_mag, ().shape

(8, 9)

time: 5.95 ms (started: 2021-03-31 00:24:54 -05:00)


In [129]:
print(ex.Ybus_mag)

[[ 0.     16.      0.      0.      0.      0.     16.      0.      0.    ]
 [ 0.      0.     17.065   0.      0.      0.      0.      0.     17.065 ]
 [17.361   0.      0.     39.614  11.684  10.689   0.      0.      0.    ]
 [ 0.      0.      0.     11.684  17.764   0.      6.092   0.      0.    ]
 [ 0.      0.      0.     10.689   0.     16.419   0.      0.      5.7334]
 [ 0.     16.      0.      0.      6.092   0.     35.783  13.793   0.    ]
 [ 0.      0.      0.      0.      0.      0.     13.793  23.645   9.8522]
 [ 0.      0.     17.065   0.      0.      5.7334  0.      9.8522 32.529 ]]
time: 4.75 ms (started: 2021-03-31 00:23:07 -05:00)


In [94]:
pl_df = pd.read_csv(power_grid._get_path("pl"), header=0)

time: 8.19 ms (started: 2021-03-30 23:37:38 -05:00)


In [106]:
pl_ixs = pl_df.iloc[:, 0]
pl_vals = pl_df.iloc[:, 1]

time: 4.91 ms (started: 2021-03-30 23:41:06 -05:00)


In [108]:
np.zeros(10)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

time: 2.75 ms (started: 2021-03-30 23:45:02 -05:00)


In [107]:
pl_ixs

0    2
1    3
2    4
3    5
4    6
5    7
6    8
7    9
Name: 1, dtype: int64

time: 3.51 ms (started: 2021-03-30 23:43:26 -05:00)


In [57]:
power_grid.ql

array([[2.000e+00, 9.000e+00],
       [3.000e+00, 1.000e+02],
       [4.000e+00, 1.500e+02],
       ...,
       [2.381e+03, 0.000e+00],
       [2.382e+03, 4.000e-01],
       [2.383e+03, 2.000e+01]])

time: 6.31 ms (started: 2021-03-30 23:07:30 -05:00)


In [43]:
power_grid.pl

array([[ 2. , 21.7],
       [ 3. , 94.2],
       [ 4. , 11.2],
       [ 5. ,  0. ],
       [ 6. , 47.8],
       [ 7. ,  7.6],
       [ 8. ,  0. ],
       [ 9. , 29.5],
       [10. ,  9. ],
       [11. ,  3.5],
       [12. ,  6.1],
       [13. , 13.5],
       [14. , 14.9]])

time: 5.18 ms (started: 2021-03-30 23:06:37 -05:00)


In [44]:
power_grid.pg

array([[ 2, 40],
       [ 3,  0],
       [ 4,  0],
       [ 5,  0]])

time: 5.4 ms (started: 2021-03-30 23:06:39 -05:00)


In [45]:
power_grid.qg

array([[  2.    ,  -6.997 ],
       [  3.    ,  -6.1129],
       [  4.    , 135.67  ],
       [  5.    ,   1.2275]])

time: 5.48 ms (started: 2021-03-30 23:06:41 -05:00)


In [46]:
power_grid.v

array([[ 2.    ,  1.045 ],
       [ 3.    ,  1.01  ],
       [ 4.    ,  1.07  ],
       [ 5.    ,  1.09  ],
       [ 6.    ,  1.0941],
       [ 7.    ,  1.0873],
       [ 8.    ,  1.0526],
       [ 9.    ,  1.0903],
       [10.    ,  1.0841],
       [11.    ,  1.086 ],
       [12.    ,  1.0801],
       [13.    ,  1.0767],
       [14.    ,  1.0669]])

time: 4.55 ms (started: 2021-03-30 23:06:42 -05:00)


In [47]:
power_grid.d

array([[ 2.      , -0.085603],
       [ 3.      , -0.21446 ],
       [ 4.      , -0.18823 ],
       [ 5.      , -0.25953 ],
       [ 6.      , -0.30589 ],
       [ 7.      , -0.27082 ],
       [ 8.      , -0.16316 ],
       [ 9.      , -0.29011 ],
       [10.      , -0.2976  ],
       [11.      , -0.30378 ],
       [12.      , -0.31809 ],
       [13.      , -0.31756 ],
       [14.      , -0.31847 ]])

time: 7.47 ms (started: 2021-03-30 23:06:47 -05:00)


In [36]:
power_grid.Ybus_mag.shape

(8, 9)

time: 2.91 ms (started: 2021-03-30 22:59:17 -05:00)


In [37]:
power_grid.Ybus_theta.shape

(8, 9)

time: 5.06 ms (started: 2021-03-30 22:59:25 -05:00)
